In [3]:
import numpy as np
import cv2

In [38]:
"""
Convert number into string with n characters
- x: integer
- n: total number of characters
returns a string
"""
def num_string(x, n=3):
    val = str(10**n + x)[1:]
    return val

In [40]:
"""
Digits counter
- x: integer value
returns the number of digits
"""
def digits_counter(x):
    count = 0
    while x != 0:
        x //= 10
        count += 1
    return count

In [41]:
"""
Get background image for mask generation and subtraction
- videoname: video file path and name
- savedir: folder to save the result
- framenumber: frame to capture
writes a background image in png
"""
#
def get_background(videoname, savedir, framenumber=1):
    cap = cv2.VideoCapture(videoname)
    cap.set(cv2.CAP_PROP_POS_FRAMES, framenumber)
    success, img = cap.read()
    if success:
        fname = savedir + "/background.png"
        cv2.imwrite(fname, img)

In [42]:
"""
Process bubble video
- videofile: video file path and name
- savedir: folder to save the processed files
- roi: region of interest [xinit,xend,yinit,yend]
- maskfile: mask file path and name
- bgfile: background file path and name
- vpar: [fps, start(sec), end(sec)]
- alpha: contrast increase before and after background subtraction (generally a[0]<a[1])
- ksize: kernel size (ellipse) for morph transformation(k[0]->dilate, k[1]->erode)
- searchpar: blob detector -> [minArea, minCircularity, minConvexity, minInertiaRatio]
- cwindow: window considered to count bubbles (vertical if horiz=False)
- textpos: text info position
writes the frames with the counting and the rate in bubbles/min
returns the total count
"""
def bubble_process(videofile, savedir, roi, maskfile, bgfile, vpar, alpha=[1.5,2.0],
                   ksize=[31,11], searchpar=[10,0.1,0.7,0.01], cwindow=(500,550),
                   textpos=(580,300), horiz=False):
    
    # read mask (grayscale)
    mask = cv2.imread(maskfile,0)[roi[0]:roi[1],roi[2]:roi[3]]
    #mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # set video capture
    cap = cv2.VideoCapture(videofile)
    start_frame_number = vpar[0]*vpar[1]
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame_number)

    # read and pre process background
    background = cv2.imread(bgfile, 0)[roi[0]:roi[1],roi[2]:roi[3]]
    background = cv2.equalizeHist(background)
    background = cv2.bitwise_not(background)
    
    # checking connection
    if (cap.isOpened()==False):
        print("Error opening video stream os file")
        
    # initialize counter 
    i = 1
    count = 0
    n=0
    nframes = vpar[0]*(vpar[2]-vpar[1])
    dig = digits_counter(nframes)
    
    while(cap.isOpened() and i <= nframes):
        # frame capture
        ret, frame = cap.read()
        if ret == True:
            ## frame processing
            # grayscale image
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)[roi[0]:roi[1],roi[2]:roi[3]]
            # histogram equalize
            frame_pre = cv2.equalizeHist(frame)
            # enhance contrast (1)
            frame_pre = cv2.convertScaleAbs(frame_pre, alpha=alpha[0], beta=0)
            # invert
            frame_pre = cv2.bitwise_not(frame_pre)
            # background subtraction
            frame_pre = cv2.subtract(frame_pre, background)
            # enhance contrast (2)
            frame_pre = cv2.convertScaleAbs(frame_pre, alpha=alpha[1], beta=0)
            # masked image
            frame_pre = cv2.bitwise_and(frame_pre, frame_pre, mask=mask)
            # morphological transformation (dilate and erode)
            kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ksize[0],ksize[0]))
            frame_process = cv2.morphologyEx(frame_pre, cv2.MORPH_DILATE, kernel1)
            kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ksize[1],ksize[1]))
            frame_process = cv2.morphologyEx(frame_process, cv2.MORPH_ERODE, kernel2)
            # threshold (Otsu)
            _,frame_process = cv2.threshold(frame_process, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            # invert back
            frame_process = cv2.bitwise_not(frame_process)
        
            # bubble finding
            params = cv2.SimpleBlobDetector_Params()
            params.filterByArea = True
            params.minArea = searchpar[0]
            params.filterByCircularity = True
            params.minCircularity = searchpar[1]
            params.filterByConvexity = True
            params.minConvexity = searchpar[2]
            params.filterByInertia = True
            params.minInertiaRatio = searchpar[3]

            detector = cv2.SimpleBlobDetector_create(params)
            b = detector.detect(frame_process)

            # drawing on the frame
            frame_final = cv2.drawKeypoints(frame, b, np.array([]), (0,0,255), 
                                   cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
            # counter
            if horiz == False:
                direction = 0
                L1a = (cwindow[0],0)
                L1b = (cwindow[0],frame.shape[0])
                L2a = (cwindow[1],0)
                L2b = (cwindow[1],frame.shape[0])
            else:
                direction = 1
                L1a = (0, cwindow[0])
                L1b = (frame.shape[1], cwindow[0])
                L2a = (0, cwindow[1])
                L2b = (frame.shape[1], cwindow[1])
                
            valid = [v.pt[direction] > cwindow[0] and v.pt[direction] < cwindow[1] for v in b]
            
            if sum(valid) != n:
                if sum(valid) > n:
                    count = count + sum(valid) - n
                    n = sum(valid)
                else:
                    n = sum(valid)

            # elapsed time (minutes)
            t = (i - 1)*(1/(vpar[0]*60))
            if t == 0:
                t = 1

            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame_final,"count = " + str(count) + " bubbles", textpos, font, 0.8,(0,0,0),2)
            cv2.putText(frame_final,"rate = " + str(int(np.round(count/t,0))) + " bubbles/min", (textpos[0],textpos[1]+30), font, 0.8,(0,0,0),2)
            cv2.line(frame_final, L1a, L1b, (0,255,0), thickness=2)
            cv2.line(frame_final, L2a, L2b, (0,255,0), thickness=2)
            
            # Saving frames
            fname_final = savedir+"/frame_process-"+num_string(i,dig)+".png"
            cv2.imwrite(fname_final, frame_final)

        
            # Teclar q para sair
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
        i = i+1

    cap.release()
    cv2.destroyAllWindows()
    return count



In [32]:
get_background("videos/sexta-02.MP4", "teste/", framenumber=1)

In [43]:
bubble_process("videos/sexta-02.MP4", "teste/", [600,1550,270,660], "teste/mask.png", 
               "teste/background.png", vpar=[240,15,20], alpha=[1.5,2.0],
                   ksize=[31,11], searchpar=[10,0.1,0.7,0.01], cwindow=(500,530),
                   textpos=(50,880), horiz=True)